# 패키지 설치

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경 변수를 불러옵니다.

# api_key = os.getenv("OPENAI_API_KEY")
# print(f"API Key: {api_key}")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [2]:
# print('hello world')
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores.faiss import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

prompt = PromptTemplate.from_template("Question: {question}\nAnswer:")

Question = '당신은 누구십니까?'

response = llm.invoke(prompt.format(question=Question))

In [4]:
response

AIMessage(content='저는 인공지능 챗봇입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 25, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4d465ffc-8806-4f67-bedb-2b6fa95c36bc-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 40})

In [5]:
import requests

urls = {
    "namu_crawler.py":"https://raw.githubusercontent.com/lymanstudio/proj_artist_info_gen/main/namu_crawler.py",
    "namu_loader.py":"https://raw.githubusercontent.com/lymanstudio/proj_artist_info_gen/main/namu_loader.py"
}

for key, val in urls.items():    
    r = requests.get(val)

    # 다운로드한 파일을 현재 디렉토리에 저장
    with open(key, "w", encoding='utf-8') as file:
        file.write(r.text)


In [6]:
# downloaded_module에서 클래스를 임포트
from namu_crawler import NamuCrawler

# MyClass의 인스턴스를 생성하고 메서드를 호출
my_instance = NamuCrawler("https://namu.wiki/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8", 3)
my_instance.construct_toc()
my_instance.get_doc_title()

'프로미스나인'

In [7]:
my_instance.get_content_heading('s-6')

(('6.', '활동'),
 '/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8/%ED%99%9C%EB%8F%99')

In [ ]:
my_instance.construct_toc()
my_instance.print_toc()

In [93]:
from namu_loader import NamuLoader
my_instance2 = NamuLoader("https://namu.wiki/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8", 3)

In [94]:
my_instance2.get_total_content()

TypeError: NamuLoader.get_total_content() missing 3 required positional arguments: 'parent_item', 'sub_url', and 'hop'

In [79]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://namu.wiki/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8/%EC%9D%8C%EC%95%85%20%EB%B0%A9%EC%86%A1%20%EC%A7%81%EC%BA%A0'
hop = 3 # 현재 문서가 몇번째 hop(depth)으로 크롤링 되고 있는지
session = requests.Session()
html_doc = session.get(url)
soup = BeautifulSoup(html_doc.text, 'html.parser')

In [80]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta charset="utf-8"/>
  <meta content="user-scalable=no, initial-scale=1, width=device-width, viewport-fit=cover" name="viewport"/>
  <title>
   프로미스나인/음악 방송 직캠 - 나무위키
  </title>
  <meta content="ie=edge" http-equv="x-ua-compatible"/>
  <meta content="the seed" name="generator"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="나무위키" name="application-name"/>
  <meta content="나무위키" name="msapplication-tooltip"/>
  <meta content="/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8" name="msapplication-starturl"/>
  <meta content="max-image-preview:large" name="robots"/>
  <meta content="프로미스나인/음악 방송 직캠" property="og:title"/>
  <meta content="나무위키" property="og:site_name"/>
  <meta content="//i.namu.wiki/i/jsiL7u7vGQAnq_oBSJJZsgsz8-J9H4HWaW9jgajxcuAedcRSzEogpHp3TPGKyvL9Att6ccCCSwF4IMFfVdFbF_8hRkOBlwj9Md-Mb3fqJfQbhnI1eW5o-0EfMcGNnQgErR9tWVPzS4wfV-ysQp17rQ.webp" property="og:image"/>
  <meta content="파일:Supersonic 24

In [81]:
tbls = soup.find_all('table')

In [86]:
tbls[7]

<table class="wiki-table" data-dark-style="background-color:#1c1d1f;" data-v-76cbfa86="" style="background-color:#fff; width:100%; border:2px solid #8aa9d8;"><tbody data-v-76cbfa86=""><tr data-v-76cbfa86="" style="background-color:#8aa9d8; color:#0f447e;"><td data-v-76cbfa86="" style="width:20%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">곡 제목</strong></div></td><td data-v-76cbfa86="" style="width:40%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">썸네일 이미지</strong></div></td><td data-v-76cbfa86="" style="width:20%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">음악 방송</strong></div></td><td data-v-76cbfa86="" style="width:20%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">날짜</strong></div></td></tr><tr data-v-76cbfa86=""><td colspan="4" data-v-76cbfa86="" style="text-align:center;"><div class="wiki

In [ ]:
profile_table = soup.find_all('div', class_ = 'wiki-table-wrap table-right') #얘는 profile 전용

In [16]:
profile_table

[<div class="wiki-table-wrap table-right" data-v-76cbfa86="" style="width:400px"><table class="wiki-table" data-dark-style="background-color:#1c1d1f;" data-v-76cbfa86="" style="background-color:#fff; width:100%; border:2px solid #f2f1f1;"><tbody data-v-76cbfa86=""><tr data-v-76cbfa86=""><td colspan="3" data-v-76cbfa86="" style="color:#c73b32; background-color:#f2f1f1; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><div data-v-76cbfa86="" style="margin:-10px -10px"><div class="wiki-table-wrap table-center" data-v-76cbfa86=""><table class="wiki-table" data-v-76cbfa86="" style="background-color:#f2f1f1; border:2px solid #f2f1f1;"><tbody data-v-76cbfa86=""><tr data-v-76cbfa86=""><td data-v-76cbfa86="" style="text-align:right;"><div class="wiki-paragraph" data-v-76cbfa86=""><span class="wiki-image-align-normal" data-v-76cbfa86="" style="height:50px;"><span class="wiki-image-wrapper" data-v-76cbfa86="" style="height: 100%;"><img alt="프로미스나인 Supersoni..." data-v-76cbfa86="

In [32]:
eles = profile_table[0].find_all('div', class_ = 'wiki-paragraph')

In [92]:
eles[40].text

'링크'

In [ ]:
#TODO
1. 전략: 불필요한 태그 삭제 후 텍스트 추출

tr과 td의 조합 > td가 중첩이 심함.
tr이 중첩된 경우된 경우
    첫 번째 tr > row
    그 뒤 tr > json 중첩 시킴
td
    첫 번째 td > key
    그 뒤 td > values

2. dt 하위 태그 삭제
3. href="#fn" 삭제
4. strong을 잘 쓰면 될 것 같은데...
99. colspan, rowspan을 어떻게 잘 쓸 수 있을지?

In [ ]:
#목표는 JSON으로 파싱
#컬럼명은 최대한 보존하는 것으로
{
    "table_name": "profile_table",
    "columns": ["결성일", "데뷔일", "데뷔 음반", "장르", "캡틴", "소속사", "레이블", "유통사", "팬덤", "응원봉", "공식사이트", "링크"],
    "data": [
        {
            "결성일": "2017년 9월 29일",
            "데뷔일": "2018년 1월 24일",
            "데뷔음반": "To. Heart",
            "장르": "K-POP, 댄스, 발라드, 어쿠스틱",
            "캡탠":"이새롬",
            "소속사":"플레디스엔터테인먼트",
            "레이블":"HYBE LABELS,YG PLUS,게펜 레코드",
            "유통사":"YG PLUS, 유니버셜 뮤직 재팬, 유니버셜 뮤직그룹",
            "팬덤":"flover(플로버)",
            "응원봉":"플로봉",
            "공식사이트":"한국,일본",
            "링크":"https://~~~"
        }
    ]
}

In [89]:
def simplify_html(html_content):
    # 파서에 HTML 내용 전달
    soup = BeautifulSoup(html_content, 'html.parser')

    # 불필요한 div 태그 제거
    for div in soup.find_all('div'):
        # 디자인 목적의 div만 제거 (단, 필요 시 보존)
        if div.get('style') and not div.get('class'):
            div.unwrap()

    # 중첩된 span 태그도 제거
    for span in soup.find_all('span'):
        span.unwrap()
    
    # 중첩된 img 태그도 제거
    for span in soup.find_all('img'):
        span.unwrap()
        
    # 제거하고자 하는 텍스트
    stopwords = ["행정구","속령"]

    # 특정 텍스트가 포함된 모든 태그 찾기
    for word in stopwords:
        tags_to_remove = soup.find_all(string=lambda text: word in text)

        # 찾은 태그들 제거
        for tag in tags_to_remove:
            tag.parent.decompose()
    
    return str(soup).replace('\n', '')

html_snippet = simplify_html(html_origin)

In [40]:
from html2image import Html2Image
hti = Html2Image()

# HTML string containing only the part you want to convert
# html_snippet = profile_table

# Convert the HTML snippet to an image
hti.screenshot(html_str=html_snippet, save_as='specific_part.png')

['c:\\Users\\sanghyoon\\Documents\\Github\\idol_info_finder\\idol_info_finder-2\\specific_part.png']

In [90]:
html_snippet

'<table class="wiki-table" data-dark-style="background-color:#1c1d1f;" data-v-76cbfa86="" style="background-color:#fff; width:100%; border:2px solid #8aa9d8;"><tbody data-v-76cbfa86=""><tr data-v-76cbfa86="" style="background-color:#8aa9d8; color:#0f447e;"><td data-v-76cbfa86="" style="width:20%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">곡 제목</strong></div></td><td data-v-76cbfa86="" style="width:40%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">썸네일 이미지</strong></div></td><td data-v-76cbfa86="" style="width:20%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">음악 방송</strong></div></td><td data-v-76cbfa86="" style="width:20%; text-align:center;"><div class="wiki-paragraph" data-v-76cbfa86=""><strong data-v-76cbfa86="">날짜</strong></div></td></tr><tr data-v-76cbfa86=""><td colspan="4" data-v-76cbfa86="" style="text-align:center;"><div class="wik

In [91]:
from bs4 import BeautifulSoup

# Sample HTML with a table containing merged rows
html = html_snippet

# Parse the HTML
soup = BeautifulSoup(html, 'html.parser')

# Find the table
table = soup.find('table')

# Initialize variables
rows = table.find_all('tr')
parsed_data = []
rowspans = {}  # Track cells with rowspan

# Iterate through rows
for row_idx, row in enumerate(rows):
    cells = row.find_all('td')
    parsed_row = []
    col_idx = 0

    for cell in cells:
        # Handle rowspans from previous rows
        while col_idx in rowspans and rowspans[col_idx] > 0:
            parsed_row.append(parsed_data[-1][col_idx])
            rowspans[col_idx] -= 1
            col_idx += 1

        # Get cell value
        cell_value = cell.get_text(strip=True)

        # Handle rowspan in current cell
        rowspan = int(cell.get('rowspan', 1))
        if rowspan > 1:
            rowspans[col_idx] = rowspan - 1

        parsed_row.append(cell_value)
        col_idx += 1

    # Fill in any remaining rowspan cells
    while col_idx in rowspans and rowspans[col_idx] > 0:
        parsed_row.append(parsed_data[-1][col_idx])
        rowspans[col_idx] -= 1
        col_idx += 1

    parsed_data.append(parsed_row)

# Print the parsed data
for row in parsed_data:
    print(row)

['곡 제목', '썸네일 이미지', '음악 방송', '날짜']
['The 1st Mini AlbumTo. Heart']
['SUB환상속의 그대', '', '', '2018.01.25MPD직캠']
['TITLETo Heart', '', '', '2018.01.25MPD직캠']
['TITLETo Heart', '', '', '2018.01.27예능연구소']
['TITLETo Heart', '', '', '2018.02.01MPD직캠']
['TITLETo Heart', '', '', '2018.02.03예능연구소']
['TITLETo Heart', '', '', '2018.02.09Fancam']
['TITLETo Heart', '', '', '2018.02.18무대 직캠']
['TITLETo Heart', '', '', '2018.02.24예능연구소']


In [1]:
from namu_crawler import NamuCrawler

# MyClass의 인스턴스를 생성하고 메서드를 호출
my_instance = NamuCrawler("https://namu.wiki/w/%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8", 0)
my_instance.construct_toc()
my_instance.get_doc_title()

'프로미스나인'

In [2]:
my_instance.construct_table_u_llm(my_instance.soup.find('table'))

AttributeError: 'NamuCrawler' object has no attribute 'construct_table_u_llm'